# OpenAI function calling with Elasticsearch

[Function calling](https://platform.openai.com/docs/guides/function-calling) in OpenAI refers to the capability of AI models to interact with external functions or APIs, allowing them to perform tasks beyond text generation. This feature enables the model to execute code, retrieve information from databases, interact with external services, and more, by calling predefined functions.

In this notebook we’re going to create two function:  
`fetch_from_elasticsearch()` - Fetch data from Elasticsearch using natural language query.   
`weather_report()` - Fetch a weather report for a particular location.

We'll integrate function calling to dynamically determine which function to call based on the user's query and generate the necessary arguments accordingly.

# Setup

### Elastic

Create an [Elastic Cloud deployment](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud) to get all Elastic credentials.  
`ES_API_KEY`: [Create](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key) an API key.  
`ES_ENDPOINT`: [Copy](https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id) endpoint of Elasticsearch.

### Open AI

`OPENAI_API_KEY`: Setup an [Open AI account and create a secret key](https://platform.openai.com/docs/quickstart).  
`GPT_MODEL`: We’re going to use the `gpt-4o` model but you can check [here](https://platform.openai.com/docs/guides/function-calling) which model is being supported for function calling.

### Open-Meteo API

We will use the [Open-Meteo API](https://open-meteo.com/). Open-Meteo is an open-source weather API and offers free access for non-commercial use. No API key required. 

`OPEN_METEO_ENDPOINT`: `https://api.open-meteo.com/v1/forecast`

### Sample Data
After creating Elastic cloud deployment, Let’s [add sample flight data](https://www.elastic.co/guide/en/kibana/8.13/get-started.html#gs-get-data-into-kibana) on kibana. Sample data will be stored into the `kibana_sample_data_flights` index.

### Install depndencies

```sh
pip install openai
```

## Import packages

In [ ]:
from openai import OpenAI
from getpass import getpass
import json
import requests

## Add Credentials

In [ ]:
OPENAI_API_KEY = getpass("OpenAI API Key:")
client = OpenAI(
    api_key=OPENAI_API_KEY,
)
GPT_MODEL = "gpt-4o"

ES_API_KEY = getpass("Elastic API Key:")
ES_ENDPOINT = input("Elasticsearch Endpoint:")
ES_INDEX = "kibana_sample_data_flights"

OPEN_METEO_ENDPOINT = "https://api.open-meteo.com/v1/forecast"

## Functions to get data from Elasticsearch

### Get Index mapping

In [ ]:
def get_index_mapping():

    url = f"""{ES_ENDPOINT}/{ES_INDEX}/_mappings"""

    headers = {
        "Content-type": "application/json",
        "Authorization": f"""ApiKey {ES_API_KEY}""",
    }

    resp = requests.request("GET", url, headers=headers)
    resp = json.loads(resp.text)
    mapping = json.dumps(resp, indent=4)

    return mapping

### Get reference document

In [ ]:
def get_ref_document():

    url = f"""{ES_ENDPOINT}/{ES_INDEX}/_search?size=1"""

    headers = {
        "Content-type": "application/json",
        "Authorization": f"""ApiKey {ES_API_KEY}""",
    }

    resp = requests.request("GET", url, headers=headers)
    resp = json.loads(resp.text)
    json_resp = json.dumps(resp["hits"]["hits"][0], indent=4)

    return json_resp

### Generate Elasticsearch Query DSL based on user query

In [ ]:
def build_query(nl_query):

    index_mapping = get_index_mapping()
    ref_document = get_ref_document()

    few_shots_prompt = """
    1. User Query - Average delay time of flights going to India
        Elasticsearch Query DSL: 
         {
          "size": 0,
          "query": {
            "bool": {
              "filter": {
                "term": {
                  "DestCountry": "IN"
                }
              }
            }
          },
          "aggs": {
            "average_delay": {
              "avg": {
                "field": "FlightDelayMin"
              }
            }
          }
        }

        2. User Query - airlines with the highest delays
        Elasticsearch Query DSL: 
         {
          "size": 0,
          "aggs": {
            "airlines_with_highest_delays": {
              "terms": {
                "field": "Carrier",
                "order": {
                  "average_delay": "desc"
                }
              },
              "aggs": {
                "average_delay": {
                  "avg": {
                    "field": "FlightDelayMin"
                  }
                }
              }
            }
          }
        }

        3. User Query - Which was the last flight that got delayed for Bangalore
        Elasticsearch Query DSL: 
        {
          "query": {
            "bool": {
              "must": [
                { "match": { "DestCityName": "Bangalore" } },
                { "term": { "FlightDelay": true } }
              ]
            }
          },
          "sort": [
            { "timestamp": { "order": "desc" } }
          ],
          "size": 1
        }
    """

    prompt = f"""
        Use below index mapping and reference document to build Elasticsearch query:

        Index mapping:
        {index_mapping}

        Reference elasticsearch document:
        {ref_document}

        Return single line Elasticsearch Query DSL according to index mapping for the below search query related to flights.:

        {nl_query}

        If any field has a `keyword` type, Just use field name instead of field.keyword.

        Just return Query DSL without REST specification (e.g. GET, POST etc.) and json markdown format (e.g. ```json)

        few example of Query DSL

        {few_shots_prompt}
        
    """

    resp = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        temperature=0,
    )

    return resp.choices[0].message.content

### Execute Query on Elasticsearch

In [ ]:
def fetch_from_elasticsearch(nl_query):

    query_dsl = build_query(nl_query)
    # print(f"""Query DSL: ==== \n\n {query_dsl}""")

    url = f"""{ES_ENDPOINT}/{ES_INDEX}/_search"""

    payload = query_dsl

    headers = {
        "Content-type": "application/json",
        "Authorization": f"""ApiKey {ES_API_KEY}""",
    }

    resp = requests.request("GET", url, headers=headers, data=payload)
    resp = json.loads(resp.text)
    json_resp = json.dumps(resp, indent=4)

    # print(f"""\n\nElasticsearch response: ==== \n\n {json_resp}""")
    return json_resp

## Function to get weather report

In [ ]:
def weather_report(latitude, longitude):

    url = f"""{OPEN_METEO_ENDPOINT}?latitude={latitude}&longitude={longitude}&current=temperature_2m,precipitation,cloud_cover,visibility,wind_speed_10m"""

    resp = requests.request("GET", url)
    resp = json.loads(resp.text)
    json_resp = json.dumps(resp, indent=4)

    # print(f"""\n\nOpen-Meteo response: ==== \n\n {json_resp}""")
    return json_resp

## Function calling

In [ ]:
def run_conversation(query):

    all_functions = [
        {
            "type": "function",
            "function": {
                "name": "fetch_from_elasticsearch",
                "description": "All flights/airline related data is stored into Elasticsearch. Call this function if receiving any query around airlines/flights.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "Exact query string which is asked by user.",
                        }
                    },
                    "required": ["query"],
                },
            },
        },
        {
            "type": "function",
            "function": {
                "name": "weather_report",
                "description": "It will return weather report in json format for given location co-ordinates.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {
                            "type": "string",
                            "description": "The latitude of a location with 0.01 degree",
                        },
                        "longitude": {
                            "type": "string",
                            "description": "The longitude of a location with 0.01 degree",
                        },
                    },
                    "required": ["latitude", "longitude"],
                },
            },
        },
    ]

    messages = []
    messages.append(
        {
            "role": "system",
            "content": "If no data received from any function. Just say there is issue fetching details from function(function_name).",
        }
    )

    messages.append(
        {
            "role": "user",
            "content": query,
        }
    )

    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
        tools=all_functions,
        tool_choice="auto",
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls

    # print(tool_calls)

    if tool_calls:

        available_functions = {
            "fetch_from_elasticsearch": fetch_from_elasticsearch,
            "weather_report": weather_report,
        }
        messages.append(response_message)

        for tool_call in tool_calls:

            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)

            if function_name == "fetch_from_elasticsearch":
                function_response = function_to_call(
                    nl_query=function_args.get("query"),
                )

            if function_name == "weather_report":
                function_response = function_to_call(
                    latitude=function_args.get("latitude"),
                    longitude=function_args.get("longitude"),
                )

            # print(function_response)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )

        second_response = client.chat.completions.create(
            model=GPT_MODEL,
            messages=messages,
        )

        return second_response.choices[0].message.content

## Ask Query

In [137]:
# Ask: details of last 10 delayed flights for Bangalore in tabular format and describe the current climate there.
i = input("Ask:")
answer = run_conversation(i)
print(answer)

Ask: details of last 10 delayed flights for Bangalore in tabular format and describe the current climate there.


### Last 10 Delayed Flights for Bangalore

| Flight Number | Origin City          | Flight Delay Type          | Flight Delay Min | Airline          | Avg Ticket Price | Origin Weather       | Destination Weather       | Status                 |
|---------------|----------------------|----------------------------|------------------|------------------|------------------|----------------------|---------------------------|------------------------|
| B2JWDRX       | Catania              | Security Delay             | 60               | Kibana Airlines  | 999.02           | Hail                 | Cloudy                    | On Time                |
| C9C7VBY       | Frankfurt am Main    | Security Delay             | 285              | Logstash Airways | 807.13           | Rain                 | Rain                      | On Time                |
| 09P9K2Z       | Paris                | Late Aircraft Delay        | 195              | Kibana Airlines  | 942.35           | Clear             